# Model Server

In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=ClassifierModel

%nuclio: setting kind to 'nuclio:serving'
%nuclio: setting 'MODEL_CLASS' environment variable


In [4]:
%nuclio config spec.build.baseImage = "mlrun/ml-models"

%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'


In [4]:
%%nuclio cmd -c
python -m pip install -U kfserving 
python -m pip install numpy cloudpickle

In [5]:
import os
from cloudpickle import load
import kfserving
import numpy as np
from typing import List

In [6]:
class ClassifierModel(kfserving.KFModel):
    def __init__(self, name: str, model_dir: str, classifier = None):
        self.name = name
        self.model_dir = model_dir
        self.ready = False
        if classifier:
            self.classifier = classifier
            self.ready = True

    def load(self):
        """Load model from KubeFlow storage.
        """
        if self.model_dir.endswith('.pkl'):
            model_file = self.model_dir
        else:
            for file in os.path.listdir(self.model_dir):
                if file.endswith('.pkl'):
                    model_file = os.pth.join(self.model_dir, file)
                    break
        self.classifier = load(open(model_file, 'rb'))
        self.ready = True

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample.
        
        :param body : A dict of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        try:
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.classifier.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")

In [7]:
# nuclio: end-code

In [8]:
from mlrun import new_model_server
fn = new_model_server('sklearn-server', model_class='ClassifierModel')
fn.spec.description = "generic sklearn model server"
fn.metadata.categories = ['serving', 'models']
fn.metadata.labels = {'author': 'yaronh'}
print(fn.to_yaml())
fn.export()

kind: remote
metadata:
  name: sklearn-server
  project: default
  labels:
    author: yaronh
  categories:
  - serving
  - models
spec:
  command: ''
  args: []
  image: ''
  description: generic sklearn model server
  volumes: []
  volume_mounts: []
  env:
  - name: MODEL_CLASS
    value: ClassifierModel
  - name: ENABLE_EXPLAINER
    value: 'False'
  config:
    spec.triggers.http:
      kind: http
      maxWorkers: 8
      attributes:
        ingresses: {}
      annotations: {}
  base_spec:
    apiVersion: nuclio.io/v1
    kind: nuclio:serving
    metadata:
      annotations:
        nuclio.io/generated_by: function generated at 06-04-2020 by admin from /User/ml/functions/model_server/model_server.ipynb
      labels: {}
      name: sklearn-server
    spec:
      build:
        baseImage: mlrun/ml-models
        commands:
        - python -m pip install -U kfserving
        - python -m pip install numpy cloudpickle
        functionSourceCode: IyBHZW5lcmF0ZWQgYnkgbnVjbGlvLmV4cG9ydC5O